### Portail meteo.data - Carte des postes météorologiques de Météo-France (Métropole & outre-mer)
Une connexion internet est nécessaire pour accéder au fichier JSON et au fond de carte

Le fichier JSON de Météo-France contient les noms du poste et de la commune, et l'alitude<br>

data : https://meteo.data.gouv.fr/ (6 min, horaire, quotidien, mensuel)<br>
Fiche PDF des postes : https://www.data.gouv.fr/fr/datasets/r/bee4b0c7-260a-40fe-b463-ed5631d6dc39 (paramètres et périodes de mesure)

https://github.com/loicduffar


##### Lecture du fichier JSON 

In [1]:
# Author:  L. Duffar
# Date:    2023-11-18
# python 3.8.12
# Description:  Extraction des données du fichier JSON des stations météo de Météo-France
import pandas as pd
import os
import datetime
import json
import requests
import sys
import plotly.express as px

# ================ Personalisation ====================
# Chemin d'enregistrement du fichier de sortie
folder_out= r"X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base"

# Chemin d'accès au fichier local JSON +++++++++++++ INUTILE TANT QUE LE FICHIER EN LIGNE EST DISPONIBLE
# folder_in= r"X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base"
# Nom du fichier JSON des stations
# file_meta= 'fiches.json'

# ================ Extrait du fichier JSON ====================
# {
#     "crs": {
#         "type": "name",
#         "properties": {
#             "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
#         }
#     },
#     "type": "featureCollection",
#     "features": {
#         "0": {
#             "geometry": {
#                 "type": "Point",
#                 "coordinates": [
#                     5.669,
#                     46.278167
#                 ]
#             },
#             "type": "Feature",
#             "properties": {
#                 "COMMUNE": "ARBENT",
#                 "NUM_POSTE": "01014002",
#                 "ALTI": 534,
#                 "NOM_USUEL": "ARBENT",
#                 "LAT_DG": 46.278167,
#                 "NUM_DEP": 1,
#                 "LON_DG": 5.669,
#                 "ficheClimComplete": 0,
#                 "ficheClimReduite": 1
#             }
#         },

# ================ Traitement ====================

# Ouverture du fichier JSON en ligne
req= requests.get('https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922')
if req.status_code == 200:
    data= req.json()
else:
    print('la requête a échoué avec le code : ', req.status_codes)
    sys.exit() # interrompt le script

# Ouverture du fichier JSON en local
# with open(os.path.join(folder_in, file_meta), encoding='utf-8') as json_file:
#     data = json.load(data)

# Création d'un dataframe
df = pd.DataFrame(data['features']).T

# Ajout de colonnes avec les champs de properties
df['lat'] = df['geometry'].apply(lambda x: x['coordinates'][1]).astype(float)
df['lon'] = df['geometry'].apply(lambda x: x['coordinates'][0]).astype(float)
df['nom_usuel']= df['properties'].apply(lambda x: x['NOM_USUEL'])
df['num_poste']= df['properties'].apply(lambda x: x['NUM_POSTE'])
df['commune'] = df['properties'].apply(lambda x: x['COMMUNE'])
df['ficheClimComplete'] = df['properties'].apply(lambda x: x['ficheClimComplete'])
df['ficheClimReduite'] = df['properties'].apply(lambda x: x['ficheClimReduite'])
df['alti'] = df['properties'].apply(lambda x: x['ALTI'])

# affiche la date et l'heure de l'extraction
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
df

2023-12-19 09:19


,geometry,type,properties,lat,lon,nom_usuel,num_poste,commune,ficheClimComplete,ficheClimReduite,alti
0,"{'type': 'Point', 'coordinates': [5.669, 46.27...",Feature,"{'COMMUNE': 'ARBENT', 'NUM_POSTE': '01014002',...",46.278167,5.669000,ARBENT,01014002,ARBENT,0,1,534
1,"{'type': 'Point', 'coordinates': [5.106667, 45...",Feature,"{'COMMUNE': 'BALAN', 'NUM_POSTE': '01027003', ...",45.833000,5.106667,BALAN_AERO,01027003,BALAN,0,0,196
2,"{'type': 'Point', 'coordinates': [4.9045, 46.1...",Feature,"{'COMMUNE': 'BANEINS', 'NUM_POSTE': '01028001'...",46.122000,4.904500,BANEINS,01028001,BANEINS,0,1,243
3,"{'type': 'Point', 'coordinates': [5.814167, 46...",Feature,"{'COMMUNE': 'VALSERHONE', 'NUM_POSTE': '010330...",46.086500,5.814167,BELLEGARDE,01033002,VALSERHONE,1,1,350
4,"{'type': 'Point', 'coordinates': [5.688, 45.76...",Feature,"{'COMMUNE': 'BELLEY', 'NUM_POSTE': '01034004',...",45.769333,5.688000,BELLEY,01034004,BELLEY,0,1,330
...,...,...,...,...,...,...,...,...,...,...,...
2387,"{'type': 'Point', 'coordinates': [165.002167, ...",Feature,"{'COMMUNE': 'POYA', 'NUM_POSTE': '98827002', '...",-21.318167,165.002167,NEPOUI,98827002,POYA,1,1,82
2388,"{'type': 'Point', 'coordinates': [165.2545, -2...",Feature,"{'COMMUNE': 'TOUHO', 'NUM_POSTE': '98830002', ...",-20.789333,165.254500,TOUHO AEROD.,98830002,TOUHO,0,1,2
2389,"{'type': 'Point', 'coordinates': [166.939, -22...",Feature,"{'COMMUNE': 'YATE', 'NUM_POSTE': '98832002', '...",-22.156500,166.939000,YATE MRIE,98832002,YATE,0,1,25
2390,"{'type': 'Point', 'coordinates': [166.726333, ...",Feature,"{'COMMUNE': 'YATE', 'NUM_POSTE': '98832006', '...",-22.132667,166.726333,RIVIERE BLANCHE,98832006,YATE,0,1,171


#### Trace la carte des postes

In [2]:
# trace une carte plotly des points

fig = px.scatter_mapbox(df, 'lat', lon= 'lon', 
                        hover_name= "nom_usuel", hover_data= ["num_poste", 'alti', "commune", 'ficheClimComplete', 'ficheClimReduite'], 
                        zoom=5, height= 800, width= 900, 
                        )

now= datetime.datetime.now()
title= 'Stations Météo-France (meteo.data.gouv) ' + now.strftime("%Y-%m-%d")
fig.update_layout(title_text= title, title_x=0.5  , hoverlabel= dict(bgcolor= 'rgb(255,255,255)'))
fig.update_traces(marker=dict(color='red'))

# Ajoute une couche de avec un fichier json des départements français
fig.update_layout(mapbox_layers=[
    {
        "sourcetype": "geojson",
        # "source": "https://france-geojson.gregoiredavid.fr/repo/departements.geojson",
        "source": "https://www.data.gouv.fr/fr/datasets/r/90b9341a-e1f7-4d75-a73c-bbc010c7feeb", 
        "type": "line",
        "color": "rgba(0, 0, 0, 0.9)",
        "line": {"width": 1}
    }
])
fig.update_layout(mapbox_style="open-street-map", mapbox_zoom=5, mapbox_center = {"lat": 46.5, "lon": 2.}
                  )
# sauvegarde le fichier html
fig.write_html(os.path.join(folder_out, 'carte_stations_meteo_france.html'))

fig.show()